In [12]:
import torch
from torch import nn
import numpy as np

import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

In [13]:
# Downloading the train MNIST dataset
mnist_train = datasets.MNIST(root='./data', # Defining the path
                             download=True, # Downloading the dataset
                             train=True,  # Defining train
                             transform=ToTensor() # Transforming data into tensors
                             )

# Downloading the test MNIST dataset
mnist_test = datasets.MNIST(root='./data', 
                            download=True, 
                            train=False,
                            transform=ToTensor()
                             )

In [14]:
# Example of image
mnist_train[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [15]:
from torch.utils.data import DataLoader

# Transforming datasets into DataLoader 
train_dataloader = DataLoader(mnist_train, batch_size=32, shuffle=True)
test_dataloader = DataLoader(mnist_test, batch_size=32, shuffle=True)

In [16]:
# 1 iteration to see the type of data created with dataloader
for X, y in train_dataloader:
    print(X.shape)
    print(y.shape)
    break

torch.Size([32, 1, 28, 28])
torch.Size([32])


In [17]:
# Defining the model
model = nn.Sequential(
    nn.Linear(784, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
)

# Defining the loos function
criterion = nn.BCEWithLogitsLoss()

# Defining the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
for i in range(10):
    loss_sum = 0
    for X, y in train_dataloader:
        X = X.reshape(-1, 784) # Reshaping the input to the correct format (32x784)
        y = (y == 0).type(torch.float32).reshape(-1, 1) # Converting output to true or false (0 or not 0) and reshaping (32x1)

        optimizer.zero_grad()
        y_pred = model(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
    
    print(loss_sum)

75.85790378299134
39.97530154954438
25.288861325680045
17.72987714882538
13.331082760648542
10.507433000196215
7.521916857394558
6.107889846253954
4.93680891461516
4.38078057637828


In [21]:
# Evaluating dataset
model.eval()
with torch.no_grad():
    accurate = 0
    total = 0

    for X, y in test_dataloader:
        X = X.reshape(-1, 784) # Reshaping the input to the correct format (32x784)
        y = (y == 0).reshape(-1, 1) # Converting output to true or false (0 or not 0) and reshaping (32x1)

        y_pred = nn.functional.sigmoid(model(X)) # Calculating the model with test dataset inputs
        correct_pred = (y_pred > 0.5) == y # Comparing with the true values
        total += correct_pred.size(0) # Calculating the total of items 
        accurate += correct_pred.type(torch.int).sum().item()

    print(f"Accuracy: {(accurate/total)*100:.2f} %")


Accuracy: 99.60 %
